In [0]:
from pandas.io.json import json_normalize
import psycopg2
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import gensim, logging
from sklearn.cluster import DBSCAN
from sklearn import preprocessing, metrics
from collections import defaultdict
from gensim.models import Word2Vec

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Step 0: Read RSS to dataframe, transform dates, create 24hr buckets

In [1]:
### Step 0: read RSS to dataframe, transform dates, create 24hr buckets and 4-day buckets
def read_rss():
    database = "rssfeed"
    hostname="rssfeed.cjgj2uy1bapa.us-east-1.rds.amazonaws.com"
    port="5432" 
    userid="postgres"
    passwrd=""
    conn_string = "host="+hostname+" port="+port+" dbname="+database+" user="+userid+" password="+passwrd
    conn = psycopg2.connect(conn_string)
    conn.autocommit=True
    cursor = conn.cursor()
    sqlSelect = "SELECT * FROM rss_entities_with_pub";
    cursor.execute(sqlSelect);
    rows = cursor.fetchall();
    return rows

def read_publishers():
    publishers = pd.read_json('data/rss_feed.json', typ='series')
    publishers = publishers.to_dict()
    pub = {v: k for k, v in publishers.items()}
    return pub

def make_rssdf():
    rows = read_rss()
    pub = read_publishers()
    rssdf = pd.DataFrame(rows)
    rssdf.columns = ['publisher_code','entity','publish_time']
    rssdf["publisher"] = rssdf["publisher_code"].map(pub)  
    return rssdf

def add_24hr_bucket():
    rssdf = make_rssdf()
    rssdf['24hour_bucket'] = pd.to_datetime(
        rssdf['publish_time'],errors='coerce',format = '%Y-%m-%dT%H:%M:%S+00:00',\
        infer_datetime_format = True, cache = True)
    return rssdf

def group_entities_by_bucket():
    rssdf = add_24hr_bucket()
    rssdf['dates']=rssdf['24hour_bucket'].dt.date
    newdf = pd.DataFrame(rssdf.groupby(['dates'])['entity']\
                      .apply(", ".join)).reset_index().sort_values(by=['dates'])
    newdf['dates'] = pd.to_datetime(newdf['dates'])
    year = str(datetime.datetime.today())[0:4]
    month = str(datetime.datetime.today())[5:7]
    day = str(datetime.datetime.today())[8:10]
    newdf = newdf[newdf['dates']<datetime.date(int(year), int(month), int(day))] 
    return newdf

def CountFrequency(my_list): 
    '''
    Given a list of entities, return a dictionary of frequencies
    '''
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq


def refine_entities():
    '''
    Create a separate column of entities with frequency > 50
    '''
    #df_window, lst = aggregate_entities_4_days()
    mydf = group_entities_by_bucket()
    df_window = mydf[(mydf['dates'] >= pd.Timestamp(datetime.date(2018,1,1)))].reset_index()
    lst = []
    for i in range(len(df_window)):
        mystr = df_window.iloc[i,2]
        mylist = list(mystr.split(", ")) 
        mydict = CountFrequency(mylist)
        lst.append(mydict)
    df_window['entity_frequency'] = lst
    
    lst2 = []
    for i in range(len(lst)):
        d = lst[i]
        lst2.append({key: d[key] for key in d if d[key] > 50})
    df_window['entity_frequency_refined'] = lst2 
    del df_window['index']
    return df_window

df_window = refine_entities()
#df_window.to_csv(r'data/df_window.csv', index = False)
#df_window = pd.read_csv("data/df_window.csv")
#df_window.head()

NameError: name 'psycopg2' is not defined

In [0]:
df_sub = df_window.iloc[len(df_window)-38:len(df_window)-9,:]
df_sub

,dates,entity,entity_frequency,entity_frequency_refined
770,2020-02-10,"EDF, Pod Point, French, UK, EDF Energy, Pod Po...","{'EDF': 2, 'Pod Point': 3, 'French': 97, 'UK':...","{'French': 97, 'UK': 527, 'China': 545, 'Europ..."
771,2020-02-11,"Dave, Coventry, New Home, Property Auction, Da...","{'Dave': 15, 'Coventry': 6, 'New Home': 1, 'Pr...","{'Microsoft': 71, 'Sydney': 57, 'Becky Lynch':..."
772,2020-02-12,"Sarah, Wimbledon, Sarah Beenys Renovate, Prope...","{'Sarah': 1, 'Wimbledon': 1, 'Sarah Beenys Ren...","{'UK': 299, 'London': 159, 'US': 317, 'Delhi':..."
773,2020-02-13,"State of Revenue Operations Report, Worcesters...","{'State of Revenue Operations Report': 1, 'Wor...","{'UK': 295, 'London': 158, 'US': 219, 'Democra..."
774,2020-02-14,"Charlie Dimmock, Rich, Garden Rescue, Property...","{'Charlie Dimmock': 1, 'Rich': 1, 'Garden Resc...","{'UK': 455, 'Spain': 53, 'Kobe Bryant': 128, '..."
775,2020-02-15,"Jenni Falconer, British, UK, Fantasy Homes by ...","{'Jenni Falconer': 2, 'British': 119, 'UK': 29...","{'British': 119, 'UK': 294, 'Aberdeen': 54, 'L..."
776,2020-02-16,"London, Cornish Countryside, Property Auction,...","{'London': 229, 'Cornish Countryside': 1, 'Pro...","{'London': 229, 'European': 82, 'Scotland': 69..."
777,2020-02-17,"HappyShappy, The Calling of the Deer, Nara, Fr...","{'HappyShappy': 1, 'The Calling of the Deer': ...","{'French': 92, 'Coronavirus': 100, 'Instagram'..."
778,2020-02-18,"ABR, YEE (YEE), YEE (CURRENCY, YEE, US, DigiFi...","{'ABR': 2, 'YEE (YEE)': 1, 'YEE (CURRENCY': 1,...","{'US': 500, 'Australian': 157, 'U.S.': 322, 'T..."
779,2020-02-19,"Property Auction, Jonie, US, Jonathan, Drew Sc...","{'Property Auction': 10, 'Jonie': 1, 'US': 526...","{'US': 526, 'Spain': 82, 'Dave': 53, 'UK': 542..."


In [0]:
len(df_sub['entity_frequency_refined'][770])

167

We will work on this df_sub from this point on.

### Step 1: Detect Events

In [0]:
### Get documents 
def get_sentences_2(df_window):
    L = df_window['entity_frequency_refined'].dropna()
    sentences2 = []
    for i in range(len(L)):
        curr = list(L[i].keys())
        sentences2.append(curr)
    return sentences2

def get_sentences(df_window):
    m = [df_window['entity'][i] for i in range(len(df_window['entity']))]
    sentences = []
    for item in m:
        sentences.append(item.split(", "))
    return sentences

sentences = get_sentences(df_window)
refined_sentences = get_sentences_2(df_window)

In [0]:
### Train or load word2vec model
def train_word2vec_model(sentences):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = gensim.models.Word2Vec(min_count=1, workers=4)
    model.build_vocab(sentences, progress_per=200000)
    model.train(sentences, epochs=model.epochs, total_examples = model.corpus_count)
    #model.save('rssword2vec.model')
    #model = gensim.models.Word2Vec.load('rssword2vec.model')
    return model

def train_model():
    sentences = get_sentences(df_window)
    model = train_word2vec_model(sentences)  # train the model on all entities
    return model

def load_model():
    model = Word2Vec.load("rssword2vec.model")
    return model

#model = load_model()
model = train_model()

2020-04-05 22:35:49,448 : INFO : collecting all words and their counts
2020-04-05 22:35:49,454 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-04-05 22:35:51,545 : INFO : collected 1409269 word types from a corpus of 5861448 raw words and 808 sentences
2020-04-05 22:35:51,547 : INFO : Loading a fresh vocabulary
2020-04-05 22:35:56,989 : INFO : effective_min_count=1 retains 1409269 unique words (100% of original 1409269, drops 0)
2020-04-05 22:35:56,990 : INFO : effective_min_count=1 leaves 5861448 word corpus (100% of original 5861448, drops 0)
2020-04-05 22:36:00,688 : INFO : deleting the raw counts dictionary of 1409269 items
2020-04-05 22:36:00,717 : INFO : sample=0.001 downsamples 7 most-common words
2020-04-05 22:36:00,718 : INFO : downsampling leaves estimated 5828381 word corpus (99.4% of prior 5861448)
2020-04-05 22:36:03,988 : INFO : estimated required memory for 1409269 words and 100 dimensions: 1832049700 bytes
2020-04-05 22:36:03,989 : INFO :

In [1]:
def create_design_matrix(row, refined_sentences, model):
    a = refined_sentences[row]
    designX = []
    for i in range(len(a)):
        try:
            designX.append(model[a[i]])
        except KeyError:
            print(a[i])
            pass
    return designX, a

def standardize_design_matrix(designX):
    standardX = preprocessing.StandardScaler().fit_transform(designX)
    return standardX

designX, a = create_design_matrix(770, refined_sentences, model)
standardX_test = standardize_design_matrix(designX)
dbscan_test = DBSCAN(metric='cosine',eps=0.1, min_samples=2) 
clusters_test = dbscan_test.fit_predict(standardX_test)


NameError: name 'refined_sentences' is not defined

In [2]:
def create_design_matrix(row, refined_sentences, model):
    a = refined_sentences[row]
    designX = []
    for i in range(len(a)):
        try:
            designX.append(model[a[i]])
        except KeyError:
            print(a[i])
            pass
    return designX, a

def standardize_design_matrix(designX):
    standardX = preprocessing.StandardScaler().fit_transform(designX)
    return standardX

def argmax(s): 
    f = lambda i: s[i]
    return max(range(len(s)), key=f)

def best_eps(eps, standardX):
    s = []
    for i in range(len(eps)):
        e = eps[i]
        dbscan = DBSCAN(metric='cosine',eps=e, min_samples=2) 
        clusters = dbscan.fit_predict(standardX)
        nclus = len(set(clusters))
        if nclus < 2:
            s.append(0)
            pass
        else:
            s.append(metrics.silhouette_score(standardX, clusters))
    best = eps[argmax(s)]
    return best

def dbscan_cluster(standardX, a):
    '''
    Given a standardized design matrix, and a list of entities (a)
    Finds the best epsilon for DBSCAN, fit and predict with the best epsilon
    and return the grouping result in a dictionary
    if there's more than one group, also return the silhouette score
    '''
    # find best epsilon
    eps = [0.01,0.05,0.1,0.2,0.3,0.5,0.7,1,1.5,2,3,5]
    best = best_eps(eps, standardX)
    print(best)
    # train the model on the best epsilon
    dbscan = DBSCAN(metric='cosine',eps=best, min_samples=3) 
    clusters = dbscan.fit_predict(standardX)
    # put the grouping info in a dictionary
    testdict = dict(zip(a, clusters))
    res = defaultdict(list) 
    for key, val in sorted(testdict.items()): 
        res[val].append(key)
    # check number of groups and calculate silhouette scores
    nclus = len(set(clusters))
    if nclus > 1:
        sil = metrics.silhouette_score(standardX, clusters)
        return dict(res), sil
    else: 
        return dict(res), 0  


def aggregate(nrows, refined_sentences, model):
    '''
    Takes in a dataframe with grouped 24-hour buckets and 4-day buckets
    Trains the word2vec model on all entities
    For each 4-day time bucket, trains a DBSCAN model and detect events
    Evaluates each DBSCAN model with silhouette scores
    Returns the same dataframe with two added columns: groups (a dictionary) and silhouettes
    '''
    #df_temp = df_window.iloc[len(df_window)-nrows: len(df_window),:]
    df_temp = df_sub
    groups = []
    silhouettes = []
    for i in range(nrows):
        print(i)
        #row = len(df_window)-nrows + i  # last nrow number of rows
        row = 770 + i
        if len(refined_sentences[row]) == 1: # if only one word in this sentence, then no design matrix
            groups.append([{0: refined_sentences[row]}])
            silhouettes.append(0)
        else:
            designX, a = create_design_matrix(row, refined_sentences, model)
            if len(designX) == 0:    # if keyword not in vocabulary, then no grouping
                groups.append([{0: refined_sentences[row]}])
                silhouettes.append(0)
            else:
                standardX = standardize_design_matrix(designX)
                group, silhouette = dbscan_cluster(standardX, a)
                groups.append([group])
                silhouettes.append(silhouette)
    df_temp['groups'] = groups
    df_temp['silhouettes'] = silhouettes
    return df_temp

df_sub_clustered = aggregate(len(df_sub), refined_sentences, model) # 29 days from Feb 10 to Mar 10
df_sub_clustered

NameError: name 'df_sub' is not defined

In [0]:
df_sub_clustered = df_sub_clustered.reset_index()
df_sub_clustered = df_sub_clustered.drop(df_sub_clustered.columns[0], axis=1)
df_sub_clustered['groups'][0]

[{-1: ['',
   'ABC',
   'AP',
   'Amazon',
   'Apple',
   'Asian',
   'Beijing',
   'CNN',
   'Canada',
   'Canadian',
   'Chicago',
   'China',
   'Chinese',
   'Coronavirus',
   'Delhi',
   'England',
   'F1',
   'Facebook',
   'Google News',
   'Hollywood',
   'Hong Kong',
   'India',
   'Indian',
   'Instagram',
   'Irish',
   'Israel',
   'Japan',
   'Japanese',
   'Kobe Bryant',
   'London',
   'Los Angeles',
   'Mercedes',
   'Mexico',
   'NASA',
   'NBA',
   'NFL',
   'NYSE',
   'Netflix',
   'New Zealand',
   'Olympics',
   'Oscar',
   'Pakistan',
   'Paris',
   'Philippines',
   'Reuters',
   'Samsung',
   'Singapore',
   'South Korea',
   'Spain',
   'Super Bowl',
   'Thailand',
   'Tokyo',
   'Twitter',
   'Ukraine',
   'WWE',
   'Wales',
   'World Cup',
   'Wuhan',
   'YouTube'],
  3: ['Academy',
   'Academy Awards',
   'BJP',
   'Bangladesh',
   'Best Picture',
   'Billie Eilish',
   'Billy Porter',
   'Bong Joon Ho',
   'Bong Joon-ho',
   'Brad Pitt',
   'Charlize Theron

In [0]:
def expand_groups_to_event_columns(df, col='groups'):
    '''
    Takes the dataframe from the output of "aggregate",
    Expands the "groups" column into separate columns, each labeled as an event
    '''
    to_dict = lambda x: x[0]
    df2 = json_normalize(df[col].apply(to_dict))
    #wide_df = pd.concat([df, df2], axis=1)
    wide_df = df.reset_index(drop=True).merge(df2.reset_index(drop=True), left_index=True, right_index=True)
    return wide_df

test = expand_groups_to_event_columns(df_sub_clustered)
test

,dates,entity,entity_frequency,entity_frequency_refined,groups,silhouettes,-1,3,2,6,5,1,8,7,0,4
0,2020-02-10,"EDF, Pod Point, French, UK, EDF Energy, Pod Po...","{'EDF': 2, 'Pod Point': 3, 'French': 97, 'UK':...","{'French': 97, 'UK': 527, 'China': 545, 'Europ...","[{-1: ['', 'ABC', 'AP', 'Amazon', 'Apple', 'As...",0.138638,"[, ABC, AP, Amazon, Apple, Asian, Beijing, CNN...","[Academy, Academy Awards, BJP, Bangladesh, Bes...","[America, American, Americans, Bernie Sanders,...","[Arsenal, Barcelona, Chelsea, Liverpool, Manch...","[Australia, Australian, Sydney]","[BBC, Boris Johnson, Brexit, Britain, British,...","[California, Florida, New York, Texas]","[Dublin, Glasgow, Northern Ireland]","[English, European, France, French, German, Ge...","[Iran, Russia, Russian]"
1,2020-02-11,"Dave, Coventry, New Home, Property Auction, Da...","{'Dave': 15, 'Coventry': 6, 'New Home': 1, 'Pr...","{'Microsoft': 71, 'Sydney': 57, 'Becky Lynch':...","[{-1: ['', 'AP', 'Android', 'Asian', 'Australi...",0.102886,"[, AP, Android, Asian, Australia, Australian, ...","[BBC, Boris Johnson, Brexit, Britain, British,...","[Amazon, Apple, Google]","[European, France, French, German, Germany, It...","[Arsenal, Barcelona, Champions League, Chelsea...","[America, American, Americans, Bernie Sanders,...",NaN,NaN,"[AAP, AI, Aberdeen, Academy Awards, BJP, Bangl...","[Iran, Russia, Russian]"
2,2020-02-12,"Sarah, Wimbledon, Sarah Beenys Renovate, Prope...","{'Sarah': 1, 'Wimbledon': 1, 'Sarah Beenys Ren...","{'UK': 299, 'London': 159, 'US': 317, 'Delhi':...","[{-1: ['', 'Amazon', 'Australia', 'Australian'...",0.090483,"[, Amazon, Australia, Australian, Britain, Bri...",NaN,NaN,NaN,NaN,"[American, U.S., United States]",NaN,NaN,"[AP, Amy Klobuchar, Andrew Yang, Bernie Sander...",NaN
3,2020-02-13,"State of Revenue Operations Report, Worcesters...","{'State of Revenue Operations Report': 1, 'Wor...","{'UK': 295, 'London': 158, 'US': 219, 'Democra...","[{3: ['', 'Coronavirus', 'McLaren', 'Sajid Jav...",0.148166,"[Amazon, Australia, Australian, California, Ca...","[, Coronavirus, McLaren, Sajid Javid, Singapor...","[Boris Johnson, Britain, EU]",NaN,"[British, London, UK]","[European, France, French, German]",NaN,NaN,"[American, Americans, Bernie Sanders, Democrat...","[Chelsea, Liverpool, Manchester United]"
4,2020-02-14,"Charlie Dimmock, Rich, Garden Rescue, Property...","{'Charlie Dimmock': 1, 'Rich': 1, 'Garden Resc...","{'UK': 455, 'Spain': 53, 'Kobe Bryant': 128, '...","[{1: ['', 'Amazon', 'Facebook', 'Google', 'Ins...",0.119605,"[Kobe Bryant, NBA]",NaN,NaN,NaN,NaN,"[, Amazon, Facebook, Google, Instagram, Netfli...",NaN,NaN,"[Aberdeen, America, American, Americans, Andre...",NaN
5,2020-02-15,"Jenni Falconer, British, UK, Fantasy Homes by ...","{'Jenni Falconer': 2, 'British': 119, 'UK': 29...","{'British': 119, 'UK': 294, 'Aberdeen': 54, 'L...","[{-1: ['', 'AP', 'Amazon', 'America', 'America...",0.197034,"[, AP, Amazon, America, American, Americans, A...",NaN,NaN,NaN,NaN,"[Arsenal, Chelsea, Manchester United]",NaN,NaN,"[Aberdeen, Amanda Bynes, Andrew Barber, Bigg B...",NaN
6,2020-02-16,"London, Cornish Countryside, Property Auction,...","{'London': 229, 'Cornish Countryside': 1, 'Pro...","{'London': 229, 'European': 82, 'Scotland': 69...","[{-1: ['', 'AP', 'American', 'Americans', 'Aus...",0.156845,"[, AP, American, Americans, Australia, Austral...","[Arsenal, Chelsea, Manchester United]","[Aston Villa, Juventus, Man City]",NaN,NaN,"[Champions League, Manchester City, Tottenham]",NaN,NaN,"[Aaron Gordon, Aberdeen, Caroline Flack, Carol...",NaN
7,2020-02-17,"HappyShappy, The Calling of the Deer, Nara, Fr...","{'HappyShappy': 1, 'The Calling of the Deer': ...","{'French': 92, 'Coronavirus': 100, 'Instagram'...","[{-1: ['', 'AP', 'Amazon', 'America', 'America...",0.154290,"[, AP, Amazon, America, American, Americans, A...",NaN,"[Champions League, Manchester City, Tottenham]",NaN,NaN,"[Arsenal, Chelsea, Manchester United]",NaN,NaN,"[Aberdeen, All-Star Game, Andrew Weat

In [0]:
test.columns

Index([                   'dates',                   'entity',
               'entity_frequency', 'entity_frequency_refined',
                         'groups',              'silhouettes',
                               -1,                          3,
                                2,                          6,
                                5,                          1,
                                8,                          7,
                                0,                          4],
      dtype='object')

In [0]:
pd.set_option('display.max_columns', None)
test = test[['dates','entity','entity_frequency','entity_frequency_refined','groups','silhouettes',\
             -1,0,1,2,3,4,5,6,7,8]]
test

,dates,entity,entity_frequency,entity_frequency_refined,groups,silhouettes,-1,0,1,2,3,4,5,6,7,8
0,2020-02-10,"EDF, Pod Point, French, UK, EDF Energy, Pod Po...","{'EDF': 2, 'Pod Point': 3, 'French': 97, 'UK':...","{'French': 97, 'UK': 527, 'China': 545, 'Europ...","[{-1: ['', 'ABC', 'AP', 'Amazon', 'Apple', 'As...",0.138638,"[, ABC, AP, Amazon, Apple, Asian, Beijing, CNN...","[English, European, France, French, German, Ge...","[BBC, Boris Johnson, Brexit, Britain, British,...","[America, American, Americans, Bernie Sanders,...","[Academy, Academy Awards, BJP, Bangladesh, Bes...","[Iran, Russia, Russian]","[Australia, Australian, Sydney]","[Arsenal, Barcelona, Chelsea, Liverpool, Manch...","[Dublin, Glasgow, Northern Ireland]","[California, Florida, New York, Texas]"
1,2020-02-11,"Dave, Coventry, New Home, Property Auction, Da...","{'Dave': 15, 'Coventry': 6, 'New Home': 1, 'Pr...","{'Microsoft': 71, 'Sydney': 57, 'Becky Lynch':...","[{-1: ['', 'AP', 'Android', 'Asian', 'Australi...",0.102886,"[, AP, Android, Asian, Australia, Australian, ...","[AAP, AI, Aberdeen, Academy Awards, BJP, Bangl...","[America, American, Americans, Bernie Sanders,...","[Amazon, Apple, Google]","[BBC, Boris Johnson, Brexit, Britain, British,...","[Iran, Russia, Russian]","[Arsenal, Barcelona, Champions League, Chelsea...","[European, France, French, German, Germany, It...",NaN,NaN
2,2020-02-12,"Sarah, Wimbledon, Sarah Beenys Renovate, Prope...","{'Sarah': 1, 'Wimbledon': 1, 'Sarah Beenys Ren...","{'UK': 299, 'London': 159, 'US': 317, 'Delhi':...","[{-1: ['', 'Amazon', 'Australia', 'Australian'...",0.090483,"[, Amazon, Australia, Australian, Britain, Bri...","[AP, Amy Klobuchar, Andrew Yang, Bernie Sander...","[American, U.S., United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-13,"State of Revenue Operations Report, Worcesters...","{'State of Revenue Operations Report': 1, 'Wor...","{'UK': 295, 'London': 158, 'US': 219, 'Democra...","[{3: ['', 'Coronavirus', 'McLaren', 'Sajid Jav...",0.148166,"[Amazon, Australia, Australian, California, Ca...","[American, Americans, Bernie Sanders, Democrat...","[European, France, French, German]","[Boris Johnson, Britain, EU]","[, Coronavirus, McLaren, Sajid Javid, Singapor...","[Chelsea, Liverpool, Manchester United]","[British, London, UK]",NaN,NaN,NaN
4,2020-02-14,"Charlie Dimmock, Rich, Garden Rescue, Property...","{'Charlie Dimmock': 1, 'Rich': 1, 'Garden Resc...","{'UK': 455, 'Spain': 53, 'Kobe Bryant': 128, '...","[{1: ['', 'Amazon', 'Facebook', 'Google', 'Ins...",0.119605,"[Kobe Bryant, NBA]","[Aberdeen, America, American, Americans, Andre...","[, Amazon, Facebook, Google, Instagram, Netfli...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-02-15,"Jenni Falconer, British, UK, Fantasy Homes by ...","{'Jenni Falconer': 2, 'British': 119, 'UK': 29...","{'British': 119, 'UK': 294, 'Aberdeen': 54, 'L...","[{-1: ['', 'AP', 'Amazon', 'America', 'America...",0.197034,"[, AP, Amazon, America, American, Americans, A...","[Aberdeen, Amanda Bynes, Andrew Barber, Bigg B...","[Arsenal, Chelsea, Manchester United]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-02-16,"London, Cornish Countryside, Property Auction,...","{'London': 229, 'Cornish Countryside': 1, 'Pro...","{'London': 229, 'European': 82, 'Scotland': 69...","[{-1: ['', 'AP', 'American', 'Americans', 'Aus...",0.156845,"[, AP, American, Americans, Australia, Austral...","[Aaron Gordon, Aberdeen, Caroline Flack, Carol...","[Champions League, Manchester City, Tottenham]","[Aston Villa, Juventus, Man City]","[Arsenal, Chelsea, Manchester United]",NaN,NaN,NaN,NaN,NaN
7,2020-02-17,"HappyShappy, The Calling of the Deer, Nara, Fr...","{'HappyShappy': 1, 'The Calling of the Deer': ...","{'French': 92, 'Coronavirus': 100, 'Instagram'...","[{-1: ['', 'AP', 'Amazon', 'America', 'America...",0.154290,"[, AP, Amazon, America, American, Americans, A...","[Aberdeen, All-Star Game, Andrew Weatherall, A...","[Arsenal, Chelsea, Manchester United]","[Champions League, Manchester City, Tottenham]",NaN,Na

In [0]:
test.to_csv('data/test.csv', index = False)

In [0]:
from datetime import timedelta, datetime
pd.set_option('display.max_colwidth', -1)

def percent_overlap(a,b): 
    '''
    Takes in two lists of strings, return three floating numbers 
    in terms of percentage overlaps
    '''
    setA = set(a)
    setB = set(b)
    overlap = setA & setB
    universe = setA | setB
    if len(setA) == 0:
        result1 = 0
    else:
        result1 = float(len(overlap)) / len(setA) * 100
    if len(setB) ==  0:
        result2 = 0
    else:
        result2 = float(len(overlap)) / len(setB) * 100
    if len(universe) == 0: 
        result3 = 0
    else:
        result3 = float(len(overlap)) / len(universe) * 100
    
    return result1,result2,result3

def event_duration2(df):
    
    # initialize the events dataframe
    columns = ['features','start','end',"index_tuple"]
    events_df = pd.DataFrame(columns = columns)
    prev_group = set([np.nan])
    prev_batch = pd.DataFrame(columns = columns)
    
    events_new = pd.DataFrame(columns = columns)
    curr_indep = pd.DataFrame(columns = columns)
    events_merge = pd.DataFrame(columns = columns)
    events_left = pd.DataFrame(columns = columns)
    
    # Compare evesxnts for every two rows in the dataset
    for i in range(len(df)-3):
        flag = False
        print("i = ",i)
        R1 = df.iloc[i,:]
        R2 = df.iloc[i+1,:]
        m = [] # an n by n matrix, each entry is an overlap score 
        curr_batch = pd.DataFrame(columns = columns)
        
        events_left = events_new.copy()
        events_merge = pd.DataFrame(columns = columns)

        # try every pair between events in row1 and events in row2
        potential_indep_eve = []
        for j in range(9): 
            mm = [] # a n by 1 vector
            for k in range(9):
                try:
                    OL, OM, OR = percent_overlap(R1[j],R2[k])
                    mm.append(max(OL, OM, OR))
                except TypeError:
                    mm.append(-1)
                    pass
            # find potentially independent events    
            setmm = set(mm)
            if (setmm == {0,-1}) and (0 in mm):
                potential_indep_eve.append(j)    
            m.append(mm)
        
        #print(m)
        
        # Find the overlap between two events in two consecutive time windows
        m_arr = np.array(m)
        nonzero_indices = np.where(m_arr > 70)
        nonzero_indices = [*zip(*nonzero_indices)]
        print("nonzero_indices = ",nonzero_indices)
        
        # Create events for lone features (no match in R2)
        
        for le in range(len(potential_indep_eve)):
            
            if potential_indep_eve[le] in prev_group:
                pass
            else:
                fea = R1[potential_indep_eve[le]]
                start = R1['dates']
                end = R1['dates']
                index_tuple = (potential_indep_eve[le], np.nan)
                newd = [[fea, start, end, index_tuple[0]]]
                curr_indep = curr_indep.append(pd.DataFrame(newd, columns = columns), ignore_index=True)
      
        # Create events when there is overlap between R1 and R2 clusters
        
        if (nonzero_indices != []) and (len(nonzero_indices[0]) != 0):
            a = [*zip(*nonzero_indices)]
            curr_group = set(a[0])
            R2_fea_to_add = list(set(a[1]))
            intersection = list(curr_group & set(prev_group))
            print("curr_group = ", curr_group, "prev_group = ", prev_group)
            
            # All rows in the current batch go to new events
            if intersection == []:
                print("All rows go to new events")
                for d in range(len(nonzero_indices)):
                    index_tuple = nonzero_indices[d]
                    start = R1['dates']
                    end = R2['dates']
                    fea = R1[index_tuple[0]] + R2[index_tuple[1]]
                    newd = [[fea, start, end, index_tuple]]
                    curr_batch = curr_batch.append(pd.DataFrame(newd, columns = columns), ignore_index=True)
                    events_new = events_new.append(pd.DataFrame(newd, columns = columns), ignore_index=True)
                    
            # Some rows go to continous events, others go to new (page3)
            else:
                print("Some new events, some continous events")
                prev_to_drop = []
                
                for k in list(curr_group): # for every k
                    print("k = ",k)
                    events_to_merge = pd.DataFrame(columns = columns)
                    
                    # continous
                    if k in prev_group:
                        print("continous")
                        index_tuple = []
                        
                        for event in range(len(prev_batch)):  # for every event in the previous batch
                            if isinstance(prev_batch.iloc[event, 3], tuple): # if only one index tuple in this event
                                tup = prev_batch.iloc[event, 3][1]
                                if tup == k:
                                    events_to_merge = events_to_merge.append(prev_batch.iloc[event,:])
                                    prev_to_drop.append(event)
                            else:    # multiple index tuples in this event
                                print("multiple index")
                                for t in range(len(prev_batch.iloc[event, 3])):
                                    tup = prev_batch.iloc[event, 3][t][1]
                                    if tup == k:
                                        events_to_merge = events_to_merge.append(prev_batch.iloc[event,:])  
                                        prev_to_drop.append(event)
                        if len(events_to_merge) > 0:                
                            print(events_to_merge)
                            fea = events_to_merge['features'].to_list()
                            fea = [item for sublist in fea for item in sublist]
                            fea_to_add = [item for item in nonzero_indices if item[0] == k]
                            for p in range(len(fea_to_add)):
                                q = R2[fea_to_add[p][1]]
                                try:
                                    fea.extend(q)
                                except TypeError:
                                    pass
                            fea = list(set(fea))
                            start = min(events_to_merge['start'].to_list())
                            end = R2['dates']

                            index_tuple.append([item for item in nonzero_indices if item[0] == k])
                            index_tuple = index_tuple[0]

                            newd = [[fea, start, end, index_tuple]]
                            curr_batch = curr_batch.append(pd.DataFrame(newd, columns = columns), ignore_index=True)
                            events_merge = events_merge.append(pd.DataFrame(newd, columns = columns), ignore_index=True)  
                        else:
                            print("No event to merge")
                            pass
                            
                        prev_to_drop = list(set(prev_to_drop))
  
                    # new 
                    else:
                        print("new")
                        flat = True
                        new_eve = [i for i in range(len(nonzero_indices)) if nonzero_indices[i][0] == k]
                        for e in range(len(new_eve)):
                            index_tuple = nonzero_indices[new_eve[e]]
                            start = R1['dates']
                            end = R2['dates']
                            fea = R1[index_tuple[0]] + R2[index_tuple[1]]
                            newd = [[fea, start, end, index_tuple]]
                            events_new = events_new.append(pd.DataFrame(newd, columns = columns), ignore_index=True)
                try:
                    events_new.drop(events_new.index[[prev_to_drop]], inplace=True)
                except IndexError:
                    pass
      
         
        print("\n")
        print("events_merge: ",events_merge, "\n")
        print("events_new: ",events_new, "\n")
        print("curr_indep: ",curr_indep, "\n")
        print("events_left: ", events_left, "\n")
        
        if flag == True:
            events_df = pd.concat([events_merge, events_new, events_left, curr_indep])
        else:
            events_df = pd.concat([events_merge, events_new, curr_indep])
      
        prev_batch = curr_batch
        prev_group = R2_fea_to_add

        print("\n\n")
        
        
        
        
    return events_df
        
events_df = event_duration2(test)          

i =  0
nonzero_indices =  [(0, 6), (1, 3), (2, 1), (4, 4), (6, 5)]
curr_group =  {0, 1, 2, 4, 6} prev_group =  {nan}
All rows go to new events


events_merge:  Empty DataFrame
Columns: [features, start, end, index_tuple]
Index: [] 

events_new:                                                                                                                                                                                                                                                                                                                                                                                                                                                                          features  \
0  [English, European, France, French, German, Germany, Italian, Italy, European, France, French, German, Germany, Italian, Italy]                                                                                                                                                            


i =  5
nonzero_indices =  [(1, 3)]
curr_group =  {1} prev_group =  [1]
Some new events, some continous events
k =  1
continous
multiple index
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

continous
multiple index
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

0 2020-02-10 2020-02-25  [(4, 1)]     

events_new:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

2 2020-03-01 2020-03-02  (3, 1)      
k =  2
new
k =  3
new
k =  4
new


events_merge:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [0]:
events_df.to_csv("data/events_df.csv")

In [3]:
pd.set_option('display.max_colwidth', -1)
d = {'dates': [datetime(2019,1,12), datetime(2020,3,28), datetime(2018,4,5), datetime(2019,2,3)], 0:[['A','B'],['A','B','C'],['C','D'],['M','N']], 1:[['F'],['C','D'],['A','G'],['A','C','D']], 2:[np.nan,['B','E'],np.nan,['G','K']], 3:[['C','E'],np.nan,['M'],['P','M']]}
fakedf = pd.DataFrame(data=d)
fakedf
event_duration2(fakedf)



NameError: name 'pd' is not defined

In [0]:
fakedf

,dates,0,1,2,3
0,2019-01-12,"[A, B]",[F],NaN,"[C, E]"
1,2020-03-28,"[A, B, C]","[C, D]","[B, E]",NaN
2,2018-04-05,"[C, D]","[A, G]",NaN,[M]
3,2019-02-03,"[M, N]","[A, C, D]","[G, K]","[P, M]"


In [0]:
### Things to do: fix this duration function
### Train word2vec better so that silhouettes increase
### Select features so that silhouettes increase
### Plot all vocab in a 2d space, identify where "coronavirus" is, why is the model not able to cluster better events?


test

NameError: name 'test' is not defined